In [ ]:
from IPython.utils.tempdir import TemporaryWorkingDirectory
!pip install -U pm4py
import pm4py
from itertools import repeat
import itertools
!pip install SNAKES
!pip install snakes_utils
import snakes.plugins
snakes.plugins.load("gv", "snakes.nets", "nets")
from nets import *
import snakes_utils

from pm4py.algo.discovery.alpha import algorithm as alpha_miner_auto
from pm4py.visualization.petri_net import visualizer as pn_visualizer

import numpy as np
from IPython.display import Image 


#read the logs file    
def import_xes(file_path):
    event_logs = pm4py.read_xes(file_path)
    '''
    # for testing purposes
    start = pm4py.get_start_activities(event_logs)
    end = pm4py.get_end_activities(event_logs)
    print("start: {}\end: {}".format(start, end))
    '''
    vars = pm4py.get_variants(event_logs)
    #print(vars)
    return vars

# Implementation of Alpha algorithm   
class AlphaMiner():
  def __init__(self,events):
    self.eventLogs = events
    self.listLogs = self.get_set_of_logs()
    #self.matrix_length = len(self.listLogs)
    self.initial_transitions = self.get_start_tasks()
    self.final_transitions = self.get_end_tasks()
    self.direct = self.directly_follows()
    self.causal = self.causality()
    self.parallel = self.parallelism()
    self.no_direct = self.no_directly_follows()
  
  def get_set_of_logs(self):
    listLogs = set()
    for key in self.eventLogs:
      for task in key:
        listLogs.add(task)
    return listLogs

  #set of start events
  def get_start_tasks(self):
    start_tasks = set()
    for key in self.eventLogs:
      start_tasks.add(key[0])
    return start_tasks

  #set of end events
  def get_end_tasks(self):
    end_tasks = set()
    for key in self.eventLogs:
      end_tasks.add(key[-1])
    return end_tasks

# Log-based ordering relations

  # >
  def directly_follows(self):
    direct_set = set()
    for event in self.eventLogs:
      for p1, p2 in zip(event, event[1:]):
        direct_set.add((p1,p2))
    return direct_set
  
  # ->
  def causality(self):
    causal_set = set()
    for couple in self.direct:
      # reverse couple is not in directly-follows set
      if couple[::-1] not in self.direct:
        causal_set.add(couple)
    return causal_set

  # #
  # we look at all permutations and possible couples of events and select those that
  # are not included in any other set
  def no_directly_follows(self):
    no_follows = set()
    all_combinations = itertools.permutations(self.listLogs, 2)
    for couple in all_combinations:
      if couple not in self.parallel and couple not in self.causal and couple[::-1] not in self.causal:
        no_follows.add(couple)
    return no_follows

  # ||
  # parallel if a couple and its inverse are in directly-follows set
  def parallelism(self):
    parallel_set = set()
    for couple in self.direct:
      if couple[::-1] in self.direct:
        parallel_set.add(couple)
    return parallel_set

  # generate Petri Net 
  def create_model(self):

    # create petri net
    n = PetriNet('Petri Net')

    # create start/end places
    n.add_place(Place('start'))
    n.add_place(Place('end'))

    # create rest of places
    for couple in self.causal:
      n.add_place(Place(couple))

    # create transitions
    for task in self.listLogs:
      n.add_transition(Transition(task))

    for pl in n.place():
      # get both elements of place
      first = pl.name[0]
      second = pl.name[1]

      # connect start/end
      for tr in n.transition():
        if pl.name == 'start' and tr.name in self.initial_transitions:
          n.add_input(pl.name, tr.name, Variable('x'))
        if pl.name == 'end' and tr.name in self.final_transitions:
          n.add_output(pl.name, tr.name, Variable('x'))

        # connect rest of places and transitions
        if first == tr.name:
          n.add_output(pl.name, tr.name, Variable('x'))
        if second == tr.name:
          n.add_input(pl.name, tr.name, Variable('x'))

    '''    
    # differentiate XOR from AND
    for tr in n.transition():
      all_second_events = set()
      if len(tr.output()) > 1:
        split_places = tr.output()
        for place in split_places:
          all_second_events.add(place[0].name[1])
          couples = set()
        test_tuple = iter(all_second_events)
        couples = list(zip(repeat(next(test_tuple)), test_tuple)) 
        print(couples)
        #for couple in couples:
          #if couple or couple[::-1] in self.no_direct:
            #TO DO merge places
      '''
   
    # generate Petri Net
    n.draw("Implemented_petri_net.png")
    Image("Implemented_petri_net.png")


def main():
    event_logs = import_xes('sample_data/BPI_Challenge_2012.xes')
    alpha_miner = AlphaMiner(event_logs)
    #events_set = alpha_miner.get_set_of_logs()

    end_events_set = alpha_miner.get_end_tasks()
    start_events_set = alpha_miner.get_start_tasks()

    direct = alpha_miner.directly_follows()
    causal = alpha_miner.causality()
    parallel = alpha_miner.parallelism()
    no_follow = alpha_miner.no_directly_follows()
  
    print("Directly follows:", direct)
    print("Causality:", causal)
    print("Parallel:", parallel)
    print("No directly follows:", no_follow)
    
    print("Start logs:", start_events_set)
    print("End logs:", end_events_set)

    alpha_miner.create_model()
    
    # generate Petri Net with PM4PY
    event_log1 = pm4py.read_xes("sample_data/BPI_Challenge_2012.xes")
    net, initial_marking, final_marking = alpha_miner_auto.apply(event_log1)
    graph = pn_visualizer.apply(net, initial_marking, final_marking)
    #pn_visualizer.view(graph)
    pn_visualizer.save(graph, "Auto_generated_petri_net.png")
    pm4py.write_pnml(net, initial_marking, final_marking, "Auto_generated_petri_net.pnml")

if __name__ == "__main__":
    main()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


parsing log, completed traces ::   0%|          | 0/13087 [00:00<?, ?it/s]

Directly follows: {('A_APPROVED', 'W_Nabellen incomplete dossiers'), ('W_Nabellen incomplete dossiers', 'O_SELECTED'), ('W_Valideren aanvraag', 'A_CANCELLED'), ('A_PARTLYSUBMITTED', 'W_Beoordelen fraude'), ('O_DECLINED', 'W_Nabellen incomplete dossiers'), ('O_ACCEPTED', 'A_ACTIVATED'), ('W_Nabellen incomplete dossiers', 'A_ACTIVATED'), ('W_Wijzigen contractgegevens', 'W_Nabellen offertes'), ('A_REGISTERED', 'A_ACTIVATED'), ('A_CANCELLED', 'W_Completeren aanvraag'), ('A_DECLINED', 'O_DECLINED'), ('A_ACTIVATED', 'W_Valideren aanvraag'), ('A_ACCEPTED', 'A_DECLINED'), ('W_Nabellen offertes', 'O_SENT_BACK'), ('A_DECLINED', 'W_Valideren aanvraag'), ('A_SUBMITTED', 'A_PARTLYSUBMITTED'), ('O_CANCELLED', 'O_SELECTED'), ('O_DECLINED', 'A_DECLINED'), ('W_Valideren aanvraag', 'O_ACCEPTED'), ('O_ACCEPTED', 'W_Nabellen incomplete dossiers'), ('A_ACTIVATED', 'A_APPROVED'), ('W_Completeren aanvraag', 'A_DECLINED'), ('W_Nabellen incomplete dossiers', 'W_Nabellen incomplete dossiers'), ('W_Valideren aan

parsing log, completed traces ::   0%|          | 0/13087 [00:00<?, ?it/s]